In [1]:
import re
import math
import collections
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams.update({"font.size": 18,
                     "font.family": "sans-serif",
                     "figure.figsize": (20, 8),
                     "axes.facecolor": "ffffff",
                     "figure.dpi"       : 200,
                     "legend.fontsize"  : "large",
                     "figure.titlesize" : "medium",
                     "lines.linewidth": 3,
})

In [2]:
# set filenames
video_analysis_file = "data/[MUSE India] [RP Outputs] - Muse_India_Study_yt_local.csv.csv"
languages = ["Bengali", "Hindi", "Kannada", "Tamil", "Telugu"]
language_analyis_files = [
    "data/[MUSE India] [Final] Language Analysis Results - bn.csv",
    "data/[MUSE India] [Final] Language Analysis Results - hi.csv",
    "data/[MUSE India] [Final] Language Analysis Results - kn.csv",
    "data/[MUSE India] [Final] Language Analysis Results - ta.csv",
    "data/[MUSE India] [Final] Language Analysis Results - te.csv"
    ]

In [3]:
# read the video analysis and language analysis dataframes
video_analysis_df = pd.read_csv(video_analysis_file, index_col=None)
language_analyis_dfs = [pd.read_csv(language_analyis_file, index_col=None) 
                        for language_analyis_file in language_analyis_files]
language_analyis_df = pd.concat(language_analyis_dfs)

In [4]:
(language_analyis_df["Transcript word count"] >= 100).sum()

1131

In [5]:
video_analysis_df["key"] = video_analysis_df["video_key"]
video_analysis_df.loc[video_analysis_df["key"].isna(), "key"] = (
    video_analysis_df.loc[video_analysis_df["key"].isna(), "Cat No."])
language_analyis_df["key"] = language_analyis_df["Video ID"].str.strip(".mp4").str.strip(".mov")

In [6]:
print(video_analysis_df.shape)
print(language_analyis_df.shape)

(1199, 120)
(1186, 15)


In [7]:
print(video_analysis_df.drop_duplicates("key", keep=False).shape)
print(language_analyis_df.drop_duplicates("key", keep=False).shape)

(1137, 120)
(1186, 15)


In [4]:
# print column names of the video analysis data
for col in video_analysis_df.columns:
    print(col)

Year
Rank
Program name
Channel
Program Theme
Program Genre
Programme Language
# of episodes
rat%/AP
Daily Avg Rch%
Daily Avg Rch'000
Ats(viewer)
Program duration
Cat No.
YouTu.be link
video_key
Notes
frames_analyzed
(female, [0, 18), [-inf, 1.1))
(female, [0, 18), [1.1, 2.1))
(female, [0, 18), [2.1, 3.1))
(female, [0, 18), [3.1, 4.1))
(female, [0, 18), [4.1, 5.1))
(female, [0, 18), [5.1, 6.1))
(female, [0, 18), [6.1, 7.1))
(female, [0, 18), [7.1, 8.1))
(female, [0, 18), [8.1, 9.1))
(female, [18, 33), [-inf, 1.1))
(female, [18, 33), [1.1, 2.1))
(female, [18, 33), [2.1, 3.1))
(female, [18, 33), [3.1, 4.1))
(female, [18, 33), [4.1, 5.1))
(female, [18, 33), [5.1, 6.1))
(female, [18, 33), [6.1, 7.1))
(female, [18, 33), [7.1, 8.1))
(female, [18, 33), [8.1, 9.1))
(female, [18, 33), [9.1, 10.1))
(female, [33, 60), [-inf, 1.1))
(female, [33, 60), [1.1, 2.1))
(female, [33, 60), [2.1, 3.1))
(female, [33, 60), [3.1, 4.1))
(female, [33, 60), [4.1, 5.1))
(female, [33, 60), [5.1, 6.1))
(female, [33, 

In [5]:
# print years
collections.Counter(video_analysis_df["Year"])

Counter({2018: 225, 2019: 231, 2020: 250, 2021: 228, 2022: 265})

In [6]:
# print number of unique program names
video_analysis_df["Program name"].unique().size

152

In [7]:
# check relation between program name and year
video_analysis_df[["Program name", "Year"]].groupby("Program name")["Year"].unique()

Program name
AALTA PHORING                         [2022]
ABHIYUM NANUM             [2020, 2021, 2022]
ADORINI                               [2018]
AGNISAKSHI                            [2018]
ALO CHHAYA                            [2019]
                                 ...        
VISHNU PURAN                          [2020]
YAARE NEE MOHINI          [2018, 2019, 2020]
YAMALEELA                             [2020]
YEH HAI CHAHATEIN                     [2022]
ZINDAGI MERE GHAR AANA                [2022]
Name: Year, Length: 152, dtype: object

In [8]:
# print number of unique channels
video_analysis_df["Channel"].unique().size

25

In [9]:
# print the unique channels
sorted(video_analysis_df["Channel"].unique())

['Big Magic',
 'Colors',
 'Colors Kannada',
 'Colors Rishtey',
 'DD Bharati',
 'DD National',
 'Dangal',
 'Dangal/DD National',
 'ETV Telugu',
 'STAR Jalsha',
 'STAR Maa',
 'STAR Plus',
 'STAR Utsav',
 'STAR Vijay',
 'Shemaroo TV',
 'Sony Pal',
 'Sun TV',
 'Udaya TV',
 'Zee Anmol',
 'Zee Bangla',
 'Zee Kannada',
 'Zee TV',
 'Zee TV/Zee Anmol',
 'Zee Tamil',
 'Zee Telugu']

In [10]:
# print program names broadcast in more than one channel
df = video_analysis_df[["Program name", "Channel"]].groupby("Program name")["Channel"].unique()
df[df.apply(len) > 1]

Program name
RAMAYAN      [Dangal, Dangal/DD National]
TRINAYANI        [Zee Bangla, Zee Telugu]
Name: Channel, dtype: object

In [11]:
# print distribution of program themes
collections.Counter(video_analysis_df["Program Theme"])

Counter({'DRAMA/SOAP': 1115,
         'ACTION/THRILLER': 10,
         'MYTHOLOGICAL/COSTUME DRAMAS': 69,
         'CHILDRENS PROGRAM': 5})

In [12]:
# check correspondence between program name and program theme
video_analysis_df[["Program name", "Program Theme"]].groupby("Program name")["Program Theme"].unique().apply(len).unique()

array([1])

In [13]:
# print program names broadcast in more than one language
df = video_analysis_df[["Program name", "Programme Language"]].groupby("Program name")["Programme Language"].unique()
df[df.apply(len) > 1]

Program name
TRINAYANI    [BENGALI, TELUGU]
Name: Programme Language, dtype: object

In [14]:
# print number of videos per year per language
video_analysis_df.groupby(["Year", "Programme Language"]).count()["Program name"]

Year  Programme Language
2018  BENGALI               35
      HINDI                 47
      KANNADA               49
      TAMIL                 45
      TELUGU                49
2019  BENGALI               36
      HINDI                 50
      KANNADA               45
      TAMIL                 50
      TELUGU                50
2020  BENGALI               41
      HINDI                 65
      KANNADA               50
      TAMIL                 45
      TELUGU                49
2021  BENGALI               49
      HINDI                 50
      KANNADA               49
      TAMIL                 50
      TELUGU                30
2022  BENGALI               69
      HINDI                 56
      KANNADA               39
      TAMIL                 50
      TELUGU                51
Name: Program name, dtype: int64

In [15]:
# print ratings vars
video_analysis_df[["rat%/AP", "Daily Avg Rch%", "Ats(viewer)", "Program duration"]]

,rat%/AP,Daily Avg Rch%,Ats(viewer),Program duration
0,2.48,5.00,0:14:50,0:29:52
1,2.48,5.00,0:14:50,0:29:52
2,2.48,5.00,0:14:50,0:29:52
3,2.48,5.00,0:14:50,0:29:52
4,2.48,5.00,0:14:50,0:29:52
...,...,...,...,...
1194,5.20,8.68,0:18:00,0:30:02
1195,5.20,8.68,0:18:00,0:30:02
1196,5.20,8.68,0:18:00,0:30:02
1197,5.20,8.68,0:18:00,0:30:02


In [16]:
# get average time percentage
def parse_duration(x):
    h, m, s = x.split(":")
    h, m, s = int(h), int(m), int(s)
    duration = 3600*h + 60*m + s
    return duration

video_analysis_df["Ats(viewer)(sec)"] = video_analysis_df["Ats(viewer)"].apply(lambda x: parse_duration(x))
video_analysis_df["Program duration(sec)"] = video_analysis_df["Program duration"].apply(lambda x: parse_duration(x))
video_analysis_df["Ats(viewer)%"] = (
    100 * video_analysis_df["Ats(viewer)(sec)"]/video_analysis_df["Program duration(sec)"])
video_analysis_df[["Ats(viewer)", "Program duration", "Ats(viewer)(sec)", "Program duration(sec)", "Ats(viewer)%"]]

,Ats(viewer),Program duration,Ats(viewer)(sec),Program duration(sec),Ats(viewer)%
0,0:14:50,0:29:52,890,1792,49.665179
1,0:14:50,0:29:52,890,1792,49.665179
2,0:14:50,0:29:52,890,1792,49.665179
3,0:14:50,0:29:52,890,1792,49.665179
4,0:14:50,0:29:52,890,1792,49.665179
...,...,...,...,...,...
1194,0:18:00,0:30:02,1080,1802,59.933407
1195,0:18:00,0:30:02,1080,1802,59.933407
1196,0:18:00,0:30:02,1080,1802,59.933407
1197,0:18:00,0:30:02,1080,1802,59.933407


In [17]:
# create faces data
# n_faces_arr[i, j, k, l] = number of faces of gender j and age k and skintone l in video i
gender_cats = ["male", "female"]
age_cats = ["[0, 18)", "[18, 33)", "[33, 60)", "[60, inf)"]
skintone_cats = ["[-inf, 1.1)", "[1.1, 2.1)", "[2.1, 3.1)", "[3.1, 4.1)", "[4.1, 5.1)", "[5.1, 6.1)", "[6.1, 7.1)",
                 "[7.1, 8.1)", "[8.1, 9.1)", "[9.1, 10.1)"]
n_faces_arr = np.zeros((len(video_analysis_df), len(gender_cats), len(age_cats), len(skintone_cats)), dtype=int)

for index, row in video_analysis_df.iterrows():
    for i, gender_cat in enumerate(gender_cats):
        for j, age_cat in enumerate(age_cats):
            for k, skintone_cat in enumerate(skintone_cats):
                cat = f"({gender_cat}, {age_cat}, {skintone_cat})"
                if cat in row and pd.notna(row[cat]):
                    n_faces_arr[index, i, j, k] = row[cat]

In [18]:
# check if the number of faces for gender, age, and skintone is consistent
for index, row in video_analysis_df.iterrows():
    assert n_faces_arr[index].sum() == row["faces"]
    assert all(n_faces_arr[index].sum(axis=(1, 2)) == row[["masculine_faces", "feminine_faces"]].tolist())
    assert all(n_faces_arr[index].sum(axis=(0, 2)) == row[age_cats].tolist())
    assert all(n_faces_arr[index].sum(axis=(0, 1)) == row[[f"mst_scale_{i}" for i in range(1, 11)]])

In [19]:
# create video id column
video_analysis_df["Video ID"] = video_analysis_df["video_key"]
video_analysis_df.loc[video_analysis_df["Video ID"].isna(), "Video ID"] = (
    video_analysis_df.loc[video_analysis_df["Video ID"].isna(), "Cat No."])

In [20]:
# create metadata df of video id, video key, year, program name, genre, channel, ats, ratings, reach and language
metadata_df = video_analysis_df[["Video ID", "Year", "Program name", "Programme Language", "Program Theme", "Channel", 
                                 "Ats(viewer)%", "rat%/AP", "Daily Avg Rch%"]].copy()
metadata_df.columns = ["video_id", "year", "program", "lang", "genre", "channel", "ats", "rating", "reach"]
metadata_df["video_key"] = np.arange(len(video_analysis_df))
metadata_df = metadata_df[["video_key", "video_id", "program", "year", "lang", "genre", "channel", 
                           "ats", "rating", "reach"]]

In [21]:
# create metadata and video analysis dataframe in wide and long forms
# columns are video_key, program name, year, lang, gender, age, skintone, number of faces
# each video_key will have 2 x 4 x 3 rows = 24 rows
long_data_rows = []
wide_data_rows = []
named_age_cats = ["young", "adult", "middle_aged", "old"]
named_skintone_cats = ["light", "medium", "dark"]

for index, row in metadata_df.iterrows():
    total_faces = n_faces_arr[index].sum()

    for i, gender_cat in enumerate(gender_cats):
        for j, age_cat in enumerate(named_age_cats):
            for skintone_cat in named_skintone_cats:
                if skintone_cat == "light":
                    k1, k2 = 0, 3
                elif skintone_cat == "medium":
                    k1, k2 = 3, 6
                else:
                    k1, k2 = 6, 10
                faces = n_faces_arr[index, i, j, k1 : k2].sum()
                long_data_rows.append(row.tolist() + [gender_cat, age_cat, skintone_cat, faces])
    
    gender_faces = n_faces_arr[index].sum(axis=(1, 2)).tolist()
    age_faces = n_faces_arr[index].sum(axis=(0, 2)).tolist()
    skintone_faces = n_faces_arr[index].sum(axis=(0, 1))
    skintone_faces = [skintone_faces[:3].sum(), skintone_faces[3:6].sum(), skintone_faces[6:].sum()]
    wide_data_rows.append(row.tolist() + gender_faces + age_faces + skintone_faces + [total_faces])

long_data_df = pd.DataFrame(long_data_rows, columns=metadata_df.columns.tolist() 
                            + ["gender", "age", "skintone", "faces"])
long_data_df.to_csv("video_only_long_form.csv", index=False)

wide_data_df = pd.DataFrame(wide_data_rows, columns=metadata_df.columns.tolist()
                            + ["male_faces", "female_faces", "young_faces", "adult_faces", "middle_aged_faces",
                               "old_faces", "light_faces", "medium_faces", "dark_faces", "faces"])
wide_data_df.to_csv("video_only_wide_form.csv", index=False)

In [22]:
# print language analysis column
for col in language_analyis_df.columns:
    print(col)

Language
Video ID
Controversial Topics (LLM)
Derogatory Words (LLM)
Derogatory Words (dictionary)
Person names
Transcript word count
Transcript unique word count
Transcript non-stopword count
Transcript unique non-stop word count
Derogatory words (dictionary) word count
Derogatory words (LLM) word count
Controversial topics (LLM) word count
Person names word count


In [23]:
# number of common keys between language analysis and video analysis dataframes
lang_ids = set(language_analyis_df["Video ID"].tolist())
video_ids = set(video_analysis_df["Video ID"].tolist())
print(f"{len(video_ids)} video ids, {len(lang_ids)} lang ids")
print(len(video_ids.intersection(lang_ids)))

1168 video ids, 1186 lang ids
975


In [24]:
# print distribution of religions and genders
religions = []
genders = []

for person_names_list_str in language_analyis_df["Person names word count"]:
    person_names_list = re.findall(r"\([^\)]+\)", person_names_list_str)
    for person_name_tuple_str in person_names_list:
        person_name_tuple = person_name_tuple_str[1:-1].split(", ")
        religions.append(person_name_tuple[1].strip("'"))
        genders.append(person_name_tuple[2].strip("'"))

print(collections.Counter(religions))
print(collections.Counter(genders))

Counter({'hindu': 14250, 'muslim': 96, 'christian': 57, 'unknown': 25})
Counter({'male': 7851, 'female': 5950, 'unknown': 389, 'unisex': 238})


In [26]:
language_analyis_df["Language"].unique()

array(['bn', 'hi', 'kn', 'ta', 'te'], dtype=object)

In [35]:
# create language wide form
rows = []
header = ["video_id", "lang", "derogatory", "controversial", 
          "male_person_name", "female_person_name", "unisex_person_name",
          "hindu_person_name", "muslim_person_name", "christian_person_name",
          "transcript", "non_stopword_transcript"]

for _, row in language_analyis_df.iterrows():
    video_id = row["Video ID"]
    lang = row["Language"]

    if lang == "bn":
        lang = "BENGALI"
    elif lang == "hi":
        lang = "HINDI"
    elif lang == "kn":
        lang = "KANNADA"
    elif lang == "ta":
        lang = "TAMIL"
    elif lang == "te":
        lang = "TELUGU"

    # derogatory words
    derogatory_count_str = row["Derogatory words (dictionary) word count"]
    if derogatory_count_str != "None":
        derogatory_tuples_str = re.findall(r"\([^\)]+\)", derogatory_count_str[1:-1])
        derogatory_count = sum([int(tup[1:-1].split(", ")[1]) for tup in derogatory_tuples_str])
    else:
        derogatory_count = 0

    # controversial count
    controversial_count_str = row["Controversial topics (LLM) word count"]
    if controversial_count_str != "None":
        controversial_tuples_str = re.findall(r"\([^\)]+\)", controversial_count_str[1:-1])
        controversial_count = sum([int(tup[1:-1].split(", ")[1]) for tup in controversial_tuples_str])
    else:
        controversial_count = 0

    # person name count
    religions = []
    genders = []
    person_name_str = row["Person names word count"]
    person_names_list = re.findall(r"\([^\)]+\)", person_name_str)
    for person_name_tuple_str in person_names_list:
        person_name_tuple = person_name_tuple_str[1:-1].split(", ")
        religions.append(person_name_tuple[1].strip("'"))
        genders.append(person_name_tuple[2].strip("'"))
    religion_dict = collections.Counter(religions)
    gender_dict = collections.Counter(genders)
    
    hindu_count = religion_dict.get("hindu", 0)
    muslim_count = religion_dict.get("muslim", 0)
    christian_count = religion_dict.get("christian", 0)
    male_count = gender_dict.get("male", 0)
    female_count = gender_dict.get("female", 0)
    person_name_count = len(person_names_list)

    # transcript count
    transcript_count = row["Transcript word count"]
    nonstopword_transcript_count = row["Transcript non-stopword count"]

    # create wide-form row
    rows.append([video_id, lang, derogatory_count, controversial_count, person_name_count,
                 hindu_count, muslim_count, christian_count, male_count, female_count,
                 transcript_count, nonstopword_transcript_count])

lang_wide_df = pd.DataFrame(rows, columns=header)

In [42]:
lang_wide_df = (lang_wide_df
                .merge(wide_data_df[["video_id", "program", "year", "lang", "genre", "channel", "ats","rating",
                                     "reach"]]
                .drop_duplicates("video_id", keep=False), 
                how="left", on=["video_id", "lang"], suffixes=("_vd", "_ln"))
                .dropna())
lang_wide_df.to_csv("lang_only_wide_form.csv", index=False)

In [43]:
# create video-language wide form
wide_data_nodup_df = wide_data_df.drop_duplicates("video_id", keep=False).copy()
video_lang_wide_df =  wide_data_nodup_df.merge(lang_wide_df, how="inner", on=["video_id", "lang"], 
                                               suffixes=("_vd", "_ln"))
video_lang_wide_df.to_csv("video_lang_wide_form.csv", index=False)

In [45]:
lang_wide_df

,video_id,lang,derogatory,controversial,male_person_name,female_person_name,unisex_person_name,hindu_person_name,muslim_person_name,christian_person_name,transcript,non_stopword_transcript,program,year,genre,channel,ats,rating,reach
0,-PUpZlgFVlI,BENGALI,0,0,4,4,0,0,2,2,36,9,GAATCHORA,2022.0,DRAMA/SOAP,STAR Jalsha,59.477562,2.60,4.37
1,-R6Iou8twaw,BENGALI,0,0,7,7,0,0,7,0,16,5,GAATCHORA,2022.0,DRAMA/SOAP,STAR Jalsha,59.477562,2.60,4.37
2,-Y9oaa82Vxw,BENGALI,0,0,12,11,1,0,8,4,1626,855,JOY BABA LOKENATH,2018.0,DRAMA/SOAP,Zee Bangla,48.152296,3.93,8.17
3,0B1QD63DzOc,BENGALI,3,0,0,0,0,0,0,0,1840,1077,KHUKUMONI HOME DELIVERY,2021.0,DRAMA/SOAP,STAR Jalsha,62.888889,3.97,6.31
4,0qUmRtgLuPY,BENGALI,1,0,0,0,0,0,0,0,1649,888,TRINAYANI,2019.0,DRAMA/SOAP,Zee Bangla,57.865481,2.86,4.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1086,yk8GpevC7eU,TELUGU,2,0,9,8,0,1,3,6,1153,720,THAT IS MAHALAKSHMI,2018.0,DRAMA/SOAP,Zee Telugu,51.798561,6.05,11.68
1087,yrcoJULwm9A,TELUGU,1,0,9,9,0,0,5,4,1626,1147,NAA PERU MEENAKSHI,2019.0,DRAMA/SOAP,ETV Telugu,46.944444,4.62,9.84
1088,yzfIeSk5t3k,TELUGU,0,0,9,9,0,0,6,3,1360,834,PREMA ENTHA MADHURAM,2020.0,DRAMA/SOAP,Zee Telugu,57.592490,5.95,10.34
1089,zCXLrTSX910,TELUGU,1,0,10,10,0,0,4,6,1248,840,THAT IS MAHALAKSHMI,2018.0,DRAMA/SOAP,Zee Telugu,51.798561,6.05,11.68


In [46]:
# create language long form for religious person names

rows = []

for _, row in lang_wide_df.iterrows():
    for religion in ["hindu", "muslim", "christian"]:
        person_name_count = row[f"{religion}_person_name"]
        rows.append(row.tolist() + [religion, person_name_count])

lang_religion_long_df = pd.DataFrame(rows, columns=lang_wide_df.columns.tolist() + ["religion", "person_name_count"])
lang_religion_long_df.drop(columns=["hindu_person_name", "muslim_person_name", "christian_person_name"], inplace=False)
lang_religion_long_df.to_csv("lang_only_religion_long_form.csv", index=False)

# create language long form for gendered person names

rows = []

for _, row in lang_wide_df.iterrows():
    for gender in ["male", "female"]:
        person_name_count = row[f"{gender}_person_name"]
        rows.append(row.tolist() + [gender, person_name_count])

lang_gender_long_df = pd.DataFrame(rows, columns=lang_wide_df.columns.tolist() + ["gender", "person_name_count"])
lang_gender_long_df.drop(columns=["male_person_name", "female_person_name"], inplace=False)
lang_gender_long_df.to_csv("lang_only_gender_long_form.csv", index=False)